In [ ]:
import logging
import os

from hirundo import (
    GitRepo,
    HirundoCSV,
    LabelingType,
    OptimizationDataset,
    StorageConfig,
    StorageGit,
    StorageTypes,
)
from hirundo.git import GitPlainAuthBase

logger = logging.getLogger(__name__)

unique_id = os.getenv("UNIQUE_ID", "").replace(".", "-").replace("/", "-")

run_ids = [run.run_id for run in OptimizationDataset.list_runs() if run.run_id]
for run_id in run_ids:
    OptimizationDataset.cancel_by_id(run_id)
dataset_ids = [dataset.id for dataset in OptimizationDataset.list_datasets()]
for dataset_id in dataset_ids:
    OptimizationDataset.delete_by_id(dataset_id)
storage_ids = [storage_config.id for storage_config in StorageConfig.list()]
for storage_id in storage_ids:
    StorageConfig.delete_by_id(storage_id)
git_repo_ids = [git_repo.id for git_repo in GitRepo.list()]
for git_repo_id in git_repo_ids:
    GitRepo.delete_by_id(git_repo_id)

test_storage_git = StorageGit(
    repo=GitRepo(
        name=f"STT-RoboShaul-dataset{unique_id}",
        repository_url="https://huggingface.co/datasets/hirundo-io/RoboShaul",
        plain_auth=GitPlainAuthBase(
            username="blewis-hir",
            password=os.environ["HUGGINGFACE_ACCESS_TOKEN"],
        ),
    ),
    branch="main",
)
test_dataset = OptimizationDataset(
    name=f"TEST-STT-RoboShaul-dataset{unique_id}",
    labeling_type=LabelingType.SPEECH_TO_TEXT,
    language="he",
    storage_config=StorageConfig(
        name=f"STT-RoboShaul-dataset{unique_id}",
        type=StorageTypes.GIT,
        git=test_storage_git,
    ),
    data_root_url=test_storage_git.get_url("/wavs"),
    labeling_info=HirundoCSV(
        csv_url=test_storage_git.get_url("/meta.csv"),
    ),
)

test_dataset.run_optimization()

/Users/blewis/Programming/hirundo-client/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValidationError: 5 validation errors for DataOptimizationDatasetOut
labeling_type
  Field required [type=missing, input_value={'name': 'Keymakr data4 -...1500, 'path': '/data4'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
storage_integration
  Field required [type=missing, input_value={'name': 'Keymakr data4 -...1500, 'path': '/data4'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
data_root_url
  Field required [type=missing, input_value={'name': 'Keymakr data4 -...1500, 'path': '/data4'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
labeling_info
  Field required [type=missing, input_value={'name': 'Keymakr data4 -...1500, 'path': '/data4'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
status
  Field required [type=missing, input_value={'name': 'Keymakr data4 -...1500, 'path': '/data4'}}, i

In [ ]:
results = test_dataset.check_run()
results.suspects.to_csv("he-on-prem-audio-test.csv")